In [19]:
import io
import progressbar
import sys
import os

In [34]:
class LZSS:
    def __init__(self):
        self.letters_counts={}
        self.file_str = ''
        self.file_path = ''
        self.search_buffer_size = 4095
        self.lookahead_buffer_size = 7
        self.mostsig_bit = 32768
        self.output = []
        self.codes = []
        self.ones = 0
        self.zeros = 0
        self.bytes_for_ones = 2
        self.bytes_for_zeroes = 1
        self.decoded = ''
    
    def readFile(self, file_path):
        self.file_path = file_path
        self.file_str = io.open(file_path, mode="r", encoding="utf-8").read()
        self.file_str = self.file_str.replace('\n','\r\n')
        for let in self.file_str:
            if self.letters_counts.has_key(let):
                self.letters_counts[let] = self.letters_counts[let] + 1
            else:
                self.letters_counts[let] = 1
        self.codes = sorted(self.letters_counts)

    
    def getLengthMatched(self, str_1, str_2):
        i = 0
        while i < min(len(str_1), len(str_2)):
            if str_1[i] == str_2[i]:
                i += 1
            else:
                break
                
        return i
    
    def getLongestMatched(self, lookahead_buffer_index):
        matched_index = -1
        matched_length = 0
        search_buffer_index = max(lookahead_buffer_index - self.search_buffer_size, 0)
        i = search_buffer_index
        while i < lookahead_buffer_index:
            if self.file_str[i] == self.file_str[lookahead_buffer_index]:
                size_str_search = min(self.lookahead_buffer_size, lookahead_buffer_index - i)
                size_str_lookahead = min(self.lookahead_buffer_size, len(self.file_str) - lookahead_buffer_index)
                length = self.getLengthMatched(self.file_str[i: i+size_str_search], self.file_str[lookahead_buffer_index: lookahead_buffer_index + size_str_lookahead])
                if length > matched_length:
                    matched_length = length
                    matched_index = i - search_buffer_index
                    i += matched_length - 1
            i += 1
        return matched_index, matched_length
    
    def encode(self):
        i = 0
        with progressbar.ProgressBar(max_value=len(self.file_str)) as bar:
            while i < len(self.file_str):
                matched_index, matched_length = self.getLongestMatched(i)
                if matched_length > 1:
                    self.output.append(self.mostsig_bit|(matched_index << 3)|matched_length)
                    self.ones += 1
                    i += matched_length
                else:
                    self.output.append(self.codes.index(self.file_str[i]))
                    self.zeros += 1
                    i += 1
                bar.update(i)
        
        
    def decode(self):
        i = 0
        with progressbar.ProgressBar(max_value=len(self.output)) as bar:
            for item in self.output:
                if item < 128:
                    self.decoded += self.codes[item]
                else:
                    search_buffer_index = max(len(self.decoded) - self.search_buffer_size, 0)
                    real_index = ((item >> 3) & 0x0fff) + search_buffer_index
                    self.decoded += self.decoded[real_index:real_index + (item & 7)]
                i += 1
                bar.update(i)
                
    def calculateCompressionRatio(self):
        return os.path.getsize(self.file_path) * 1.0 / (self.ones * self.bytes_for_ones + self.zeros * self.bytes_for_zeroes)

In [35]:
model = LZSS()
model.readFile('omar.txt')
print model.file_str

عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر


In [36]:
model.encode()

100% (191 of 191) |######################| Elapsed Time: 0:00:00 Time:  0:00:00


In [37]:
print model.calculateCompressionRatio()

5.67796610169


In [38]:
model.decode()

100% (32 of 32) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [39]:
print model.decoded

عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر عمر
